In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from io import StringIO

# 实践中的统计
## Monsanto公司
Monsanto公司已经成为美国最大的化学公司之一，生产1000多种产品，从工业化学制品到用于铺设现代运动场所需要的人工合成地面，应有尽有  
Monsanto公司最终的成功取决于他们发现了肉鸡饲料的最优配方，使得肉鸡的饲养成本与猪、牛等其他家畜的饲养成本相比，保持在更低的水平上  
Monsanto公司运用回归分析方法模拟了肉鸡的体重y与饲料中加入的蛋氨酸数量x之间的关系。最初建立的估计的简单线性回归方程如下：  
$$\hat{y}=0.21+0.42x$$
这一估计的回归方程经检验在统计上是显著的，但是残差分析显示，曲线关系也许是描述肉鸡体重与饲料中加入蛋氨酸数量之间关系的一个更适宜的模型 5  
Monsanto公司经过进一步的研究发现，虽然少了的蛋氨酸可以提高肉鸡体重，但是当其含量达到某一水平后，继续增加蛋氨酸的含量对肉鸡体重增长的作用就变得微乎其微了  
下面估计的多元回归方程用于模拟肉鸡体重与饲料中加入的蛋氨酸数量之间的曲线关系  
$$\hat{y}=-1.89+1.32x-0.506x^2$$
利用这一回归结果，Monsanto公司能够确定在肉鸡饲料中需要添加蛋氨酸的最优数量  
在本章中，我们将通过对诸如Monsanto公司建模的例子，扩展回归分析的讨论至非线性模型 5  
建立模型是一个复杂的过程，经过这一过程，我们就能得到描述应变量与一个或一个以上自变量之间关系的估计的回归方程  
建模过程主要有两方面的问题：一是找到一个合适的描述变量之间关系的函数形式，二是选择模型应包含的自变量 5  
# 16.1 一般线性模型
假设我们采集了一个应变量y和k个自变量$x_1,x_2,\dots,x_k$的观测数据。作为建立自变量之间更复杂关系的总体框架，我们介绍含有p个自变量的**一般线性模型**的概念  
<hr />

**一般线性模型**（16-1）  
$$y=\beta_0+\beta_1z_1+\beta_2z_2+\dots+\beta_pz_p+\varepsilon$$
<hr />

在式（16-1）中，每一个自变量$z_j$都是$x_1,x_2,\dots,x_k$的函数  
最简单的情形是我们仅仅对一个变量x的函数，并且希望利用直线关系去估计y。在这种情形下，$z_1=x_1$，式（16-1）变为（16-2） 5  
$$y=\beta_0+\beta_1x_1+\varepsilon$$
在统计建模文献中，这个模型被称为具有一个预测变量的简单一阶模型  
## 16.1.1 模拟曲线关系
我们能用式（16-1）模拟形式更复杂的关系。Reynolds公司的管理人员希望对公司销售人员工作年限的长短和电子实验室天平的销售数量之间的关系进行调研  
表16-1给出了15名随机抽选的销售人员近期天平的销售数量和每一名销售人员被公司雇佣的月数  

In [ ]:
'''
5
'''
rey_data=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/Reynolds.csv')
rey_data.tail()

图16-1给出了这些数据的散点图  

In [ ]:
rey_data.plot.scatter('Months','Sales')
plt.ylim(0,400)
plt.xlim(0,120)
plt.show()

散点图表明，在被公司雇佣时间的长短和销售数量之间，可能存在一个曲线关系 5  
在考虑如何为Reynolds公司建立一个曲线关系之前，让我们首先考虑图16-2中给出的与简单一阶模型对应的Minitab输出，估计的回归方程是  

In [ ]:
'''
python	patsy	formula	~
'''
rey_formula_sap='Sales ~ Months'
rey_mod_sap=smf.ols(rey_formula_sap,rey_data).fit()

print('Sales={:.0f}+{:.2f}Months'.format(rey_mod_sap.params[0],rey_mod_sap.params[1]))

In [ ]:
rey_mod_sap.summary()

图16-3是对于的标准化残差图 5  

In [ ]:
'''
5
'''
rey_inf_sap=rey_mod_sap.get_influence()
rey_inf_sut=rey_inf_sap.summary_table()
rey_str_lv1=StringIO(rey_inf_sut.as_csv())
rey_sactter_lv1=pd.read_csv(rey_str_lv1,header=0,skiprows=[1],usecols=[2,4])
rey_sactter_lv1.columns=['hy','stdRes']
rey_sactter_lv1.plot.scatter('hy','stdRes')
plt.plot(np.array([100,400]),np.array([0,0]),linestyle='--')
plt.xlim(100,400)
plt.show()

尽管计算机输出表名：这个线性关系是显著的，并且线性关系解释了销售数量中的大部分变异性，然而标准化残差图启发我们，仍然需要一个曲线关系 5  
为了说明这是一个曲线关系，我们令式（16-1）中的$z_1=x_1,z_2=x_1^2$，于是得到模型（16-3）  
$$y=\beta_0+\beta_1x_1+\beta_2x_1^2+\varepsilon$$
这个模型被称为具有一个预测变量的二阶模型  
为了建立与这个二阶模型相对应的估计的回归方程，我们使用的统计软件包不但需要表16-1中的原始数据，而且也需要增加第二个自变量所对应的数据，即销售人员被公司雇佣月数的平方  
在图16-4中，我们给出了对应二阶模型的Minitab输出，估计的回归方程是 5  

In [ ]:
'''
python	patsy	formula	function()
'''
rey_formula_lv2='Sales ~ Months + np.power(Months,2)'
rey_mod_lv2=smf.ols(rey_formula_lv2,rey_data).fit()
print('Sales={:.1f}+{:.2f}Months+{:.4f}MonthsSq'.format(rey_mod_lv2.params[0],
                                                        rey_mod_lv2.params[1],
                                                       rey_mod_lv2.params[2]))

式中，MonthsSq代表销售人员被公司雇佣月数的平方  

In [ ]:
rey_mod_lv2.summary()

图16-5是对应的标准化残差图 5  

In [ ]:
'''
5
'''
def df_std_resid(mod):
    '''
    统计常用代码	简单线性回归	残差分析：证实模型的假定	标准化残差
    5
    '''
    inf=mod.get_influence()
    inf_sut=inf.summary_table()
    str_io=StringIO(inf_sut.as_csv())
    sactter=pd.read_csv(str_io,header=0,skiprows=[1],usecols=[2,4])
    sactter.columns=['yhat','stdRes']
    return sactter

rey_sactter_lv2=df_std_resid(rey_mod_lv2)
rey_sactter_lv2.plot.scatter('yhat','stdRes',figsize=(7,7))
plt.plot(np.array([0,350]),np.array([0,0]),linestyle='--')
plt.xlim(50,350)
plt.ylim(-3,2)
plt.show()

这张图表明：前面图16-3中的曲线模式的图形已经被消除  
在$\alpha=0.05$的显著性水平下，计算机输出表明，模型在总体上是显著的;我们还注意到，对应于自变量MonthsSq的t检验的p-值小于0.05，因此我们的结论是，在含有自变量Months的模型中，增加的自变量MonthsSq是显著的  

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.rsquared_adj
'''
print('由于R-sq={:.1%},由这个估计的回归方程给出的拟合我们应该是满意的'.format(rey_mod_lv2.rsquared_adj))

在多元回归分析中，线性这个词在术语“一般线性模型”中指的仅仅是这样一个事实：$\beta_0,\beta_1,\dots,\beta_p$全是一次幂，这并不意味着y和这些$x_i$之间存在着线性关系  
## 16.1.2 交互作用
如果原始数据集由应变量y和两个应变量$x_1,x_2$的观测值组成，在一般线性模型式（16-1）中，设$z_1=x_1,z_2=x_2,z_3=x_1^2,z_4=x_2^2和z_5=x_1x_2$，我们就能建立一个含有两个预测变量的二阶模型 5  
我们得到的模型是（16-4）  
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_1^2+\beta_4x_2^2+\beta_5x_1x_2+\varepsilon$$
在这个二阶模型中，为了说明两个变量共同作用产生的潜在影响，我们增加了一个变量$z_5=x_1x_2$。这种类型的影响称为**交互作用**  
Tyler认为，对销售量起最大影响的两个因素是单位销售价格和广告费用。为了研究这两个变量对销售量的影响，在24家做实验的商店中，与价格为2美元、2.5美元和3美元相对应的广告费用分别为50000美元和100000美元  
我们观测到的销售数量（单位：1000瓶）记录在表16-2中 5  

In [ ]:
tyl=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/Tyler.csv')
tyl.head()

表16-3是这些数据的汇总  

In [ ]:
'''
5
'''
tyl.columns=['Price','Advertising','Sales']
tyl.pivot_table(values='Sales',index='Advertising',columns='Price')

因此，当价格为2美元保持不变时，广告费用分别为50000美元和100000美元，这时候的平均销售数量之差是808-461=347  
当产品价格为2.5美元时，平均销售数量之差是646-364=282。最后，当产品的价格为3美元时，平均销售数量之差依赖于产品的销售价格  
显然，广告费用分别为50000美元和100000美元时，平均销售数量之差依赖于产品的销售价格  
换句话说，当销售价格较高时，增加广告费用的影响将要减少。上述观测结果提供了销售价格和广告费用这两个变量之间交互作用的证据  
为了给出交互作用的另一个观点，图16-6表示了6种不同的销售价格与广告费用组合的平均销售数量  
![16-6](../syn_pic/statistics_for_business_economics/16-6.png)
<center>图16-6 平均销售数量是销售价格和广告费用的函数 5</center>

换句话说，只有当我们考虑两个变量对响应变量的联合影响时，才能得出有意义的结论  
为了说明交互作用的影响，我们将利用下面的回归模型（16-5）  
$$y=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_3+\varepsilon$$
式中，y代表销售数量;$x_1$代表销售价格（美元）;$x_2$代表广告费用（1000美元）  
注意：式（16-5）反映出Tyler相信销售数量线性地依赖销售价格和广告费用，并且两个变量之间存在交互作用 5  
为了建立估计的回归方程，我们利用含有3个自变量的一般线性模型（16-6）  
$$y=\beta_0+\beta_1z_1+\beta_2z_2+\beta_3z_3+\varepsilon$$
式，$z_1$代表$x_1$;$z_2$代表$x_2$；$z_3$代表$x_1x_2$  
图16-7是对应于Tyler Personal 例子的交互作用模型的Minitab输出，得到的估计方程是  

In [ ]:
'''
5
python	patsy	formula	:
'''
tyl_formula='Sales ~ Price + Advertising + Price:Advertising'
tyl_mod=smf.ols(tyl_formula,tyl).fit()
print('Sales={:.0f}+{:.0f}Price+{:.1f}AdvExp{:.2f}PriceAdv'.format(tyl_mod.params[0],
                                                                   tyl_mod.params[1],
                                                                   tyl_mod.params[2],
                                                                   tyl_mod.params[3]))

式中，Sales代表销售数量;Price代表销售价格;AdvExp代表广告费用;PriceAdv代表交互作用项  

In [ ]:
tyl_mod.summary()

因为模型在总体上是显著的，对应于交互作用项PriceAdv的t检验的p-值是0.000,我们的结论是:在已知产品销售价格和广告费用的线性影响下，交互作用是显著的 5  
## 16.1.3 涉及应变量的变换
在说明如何利用一般线性模型模拟自变量和应变量之间各种可能存在的关系时，我们把注意力集中到包含一个或一个以上自变量的变换上  
为了解释说明时候应该对应变量进行变换，我们考虑表16-4中有关12辆汽车的英里/加仑额定值和重量的数据  

In [ ]:
mpg=pd.read_csv('../pydata-book-master/statistics_for_business_economics/ch16/MPG.csv')
mpg.head()

In [ ]:
mpg.columns=['Weight','Miles']

散点图16-8表明了在这两个变量之间存在一个负的线性关系 5  

In [ ]:
mpg.plot.scatter('Weight','Miles')
plt.ylim(0,35)
plt.xlim(2000,3800)
plt.show()

所以我们利用一个简单的一阶模型把这两个变量联系起来。图16-9是Minitab输出，得到估计的回归方程是  

In [ ]:
'''
5
'''
mpg_formula='Miles ~ Weight'
mpg_mod=smf.ols(mpg_formula,mpg).fit()
print('MPG={:.1f}{:.4f}Weight'.format(mpg_mod.params[0],
                                     mpg_mod.params[1]))

式中，MPG代表英里/加仑额定值;Weight代表汽车的重量（磅）  

In [ ]:
mpg_mod.summary()

In [ ]:
'''
python	statsmodels	Linear Regression	RegressionResults.rsquared
'''
print('模型是显著的，并且数据拟合得非常好(R-sq={:.1%})'.format(mpg_mod.rsquared))

然而，我们注意到在图16-9中，第三个观测值被识别出有一个大的标准化残差  

In [ ]:
'''
5
python	statsmodels	Statistics stats	OLSInfluence.resid_studentized
'''
mpg_inf=mpg_mod.get_influence()
mpg_inf.resid_studentized>2

In [ ]:
'''
统计常用代码	简单线性回归	残差分析：证实模型的假定	标准化残差
'''
mpg_std_df=df_std_resid(mpg_mod)
mpg_std_df.columns

图16-10是对应于一阶模型的标准化残差图  

In [ ]:
'''
5
'''
mpg_std_df.plot.scatter('yhat', 'stdRes')
plt.ylim(-2,2.5)
plt.xlim(10,35)
plt.plot(np.array([10,35]),np.array([0,0]),linestyle='--')
plt.show()

如果关于误差项的假设全都成立，我们期望看到的应是一条水平的带状图形，然而我们观察到的图形看起来不像是这种形状  
实际上，残差的变异性看来随着$\hat{y}$值的增加而增加。当显著性检验的基本假设看起来没有得到满足时，我们不能证明：得到的有关估计的回归方程统计显著性的任何结论  
非常数方程问题通常能够得到修正，修正的方法是对应变量做一个不同的比例变换。例如，用应变量的对数来代替原来的应变量，这样做的效果是压缩了应变量的数值，从而达到减少非常数方差影响的目的  
我们用英里/加仑的自然对数作应变量，在输出中的标记为LogeMPG，得到的回归结果在图16-11中，图16-12是对应的标准化残差图  

In [ ]:
'''
5
python	numpy	Mathematical functions	np.log()
python	patsy	formula	function()
'''
mpg_formula_log='np.log(Miles) ~ Weight'
mpg_mod_log=smf.ols(mpg_formula_log,mpg).fit()
mpg_mod_log.summary()

In [ ]:
mpg_inf_log=mpg_mod_log.get_influence()
mpg_inf_log.resid_studentized>2

In [ ]:
'''
5
'''
mpg_std_log=df_std_resid(mpg_mod_log)
mpg_std_log.plot.scatter('yhat', 'stdRes')
plt.ylim(-2,2)
plt.xlim(2.6,3.6)
plt.plot(np.array([2.6,3.6]),np.array([0,0]),linestyle='--')
plt.show()

我们注意一下图16-12中的标准化残差图，现在我们已经看不到楔形图了。此外，没有一个观测值被识别出有一个大的标准化残差  
因此，我们愿意推荐估计的回归方程是  

In [ ]:
print('LogeMPG={:.2f}{:.6f}Weight'.format(mpg_mod_log.params[0],
                                         mpg_mod_log.params[1]))

对于一辆重量为2500磅的汽车，为了估计的它的英里/加仑额定值，我们首先应求出英里/加仑额定值的自然对数的估计值  

In [ ]:
'''
5
python	numpy	Mathematical functions	np.exp()
'''
print('我们得到{:.1f}英里/加仑额定值'.format(np.exp(mpg_mod_log.params[0]+mpg_mod_log.params[1]*2500)))

修正非常数方程问题的另一个方法是用1/y作应变量来代替原来的应变量y。这种类型的变换叫做倒数变换  
在一般情况下，没有方法能决定，究竟是进行对数变换还是进行倒数变换效果更好，除非对两种变换都实际地试一试  
## 16.1.4 内线性的非线性模型
参数($\beta_0,\beta_1,\dots,\beta_p$)的幂次超过一次的模型称为非线性模型。指数模型与下面的回归方程有关（16-7）  
$$E(y)=\beta_0\beta_1^x$$
当应变量y随着x的增加，按一个不变的百分比，而不是一个固定的数量增加或减少时，适合应用这种模型 5  
作为一个例子，假设一种产品的销售收入y依赖于广告费用x，这个问题对应的指数模型如下所示  
$$E(y)=500\times(1.2)^x$$
$对于x=1，E(y)=500\times(1.2)^1=600;对于x=2，E(y)=500\times(1.2)^2=720;对于x=3，E(y)=500\times(1.2)^3=864;$  
注意，在这种情形下，E(y)不是按一个固定的数量增加，而是按一个不变的百分比增加;增长的百分比是20%  
我们通过对式（16-7）两边取对数，将这个非线性模型转换为一个线性模型（16-8） 5  
$$logE(y)=log\beta_0+xlog\beta_1$$
注意，如果我们设$\acute{y}=logE(y),\acute{\beta}_0=log\beta_0,\acute{\beta}_1=log\beta_1$，我们能将式（16-8）写成  
$$\acute{y}=\acute{\beta}_0+\acute{\beta}_1x$$
显然，我们能利用简单线性回归的公式求出$\acute{\beta}_0和\acute{\beta}_1$的估计量。用$\acute{b_0}$和$\acute{b_1}$表示估计量，得到下面估计的回归方程（16-9）  
$$\acute{\hat{y}}=\acute{b_0}+\acute{b_1}x$$
5  
$\acute{\hat{y}}$的反对数就是我们要求的y的预测值，或者是y的期望值  
许多非线性模型不能被转换为一个等价的线性模型。这种模型所必需的数学背景超出了本教科书的范围 5  
# 16.2 确定什么时候增加或者删除变量
在这一节我们将说明，如何而利用F检验来确定，将一个或一个以上的自变量增加到一个多元回归模型上是否适宜的问题  